In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import kruskal
from scipy.stats import rankdata
from itertools import combinations
import numpy as np
from statsmodels.stats.multitest import multipletests

file_path = "Trigger_Vividness_Data.csv"  
df = pd.read_csv(file_path)

df["Subject"] = df["Subject"].astype("category")
df["Trigger"] = df["Trigger"].astype("category")

### One-Way ANOVA (Subject vs Vividness)
model1 = smf.ols("Vividness ~ C(Subject)", data=df).fit()
anova_table1 = sm.stats.anova_lm(model1, typ=2)
p_value1 = anova_table1["PR(>F)"][0]
f_value1 = anova_table1["F"][0]

### One-Way ANOVA (Trigger vs Vividness)
model2 = smf.ols("Vividness ~ C(Trigger)", data=df).fit()
anova_table2 = sm.stats.anova_lm(model2, typ=2)
p_value2 = anova_table2["PR(>F)"][0]
f_value2 = anova_table2["F"][0]

### Two-Way ANOVA (Subject & Trigger vs Vividness)
model3 = smf.ols("Vividness ~ C(Subject) + C(Trigger) + C(Subject):C(Trigger)", data=df).fit()
anova_table3 = sm.stats.anova_lm(model3, typ=2)

###（Dunn’s Test）
def dunn_test(data, group_col, value_col):
    groups = data[group_col].unique()
    group_combinations = list(combinations(groups, 2))
    p_values = []
    
    for g1, g2 in group_combinations:
        group1 = data[data[group_col] == g1][value_col]
        group2 = data[data[group_col] == g2][value_col]
        stat, p = kruskal(group1, group2)
        p_values.append(p)
    
    reject, p_corrected, _, _ = multipletests(p_values, method='bonferroni')
    results = pd.DataFrame(group_combinations, columns=[f"{group_col}_1", f"{group_col}_2"])
    results["P-value"] = p_values
    results["Corrected P-value"] = p_corrected
    results["Significant"] = reject
    return results

posthoc_subject = dunn_test(df, "Subject", "Vividness")
posthoc_trigger = dunn_test(df, "Trigger", "Vividness")

print("One-Way ANOVA (Subject -> Vividness)")
print(f"F-value: {f_value1:.4f}, P-value: {p_value1:.4f}\n")

print("One-Way ANOVA (Trigger -> Vividness)")
print(f"F-value: {f_value2:.4f}, P-value: {p_value2:.4f}\n")

print("Two-Way ANOVA (Subject & Trigger -> Vividness)")
print(anova_table3[["F", "PR(>F)"]])

posthoc_subject.to_csv("posthoc_subject_dunn.csv", index=False)
posthoc_trigger.to_csv("posthoc_trigger_dunn.csv", index=False)


In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd

df = pd.read_csv("Trigger_Vividness_Data.csv")

def one_way_anova(data, dv, iv):
    model = smf.ols(f"{dv} ~ C({iv})", data=data).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    return anova_table

def two_way_anova(data, dv, iv1, iv2):
    model = smf.ols(f"{dv} ~ C({iv1}) * C({iv2})", data=data).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    return anova_table

def tukey_posthoc(data, dv, iv):
    tukey = pairwise_tukeyhsd(endog=data[dv], groups=data[iv], alpha=0.05)
    return tukey.summary()

anova_subject = one_way_anova(df, "Vividness", "Subject")
anova_trigger = one_way_anova(df, "Vividness", "Trigger")
anova_session = one_way_anova(df, "Vividness", "Session")

anova_subject_trigger = two_way_anova(df, "Vividness", "Subject", "Trigger")
anova_subject_session = two_way_anova(df, "Vividness", "Subject", "Session")
anova_trigger_session = two_way_anova(df, "Vividness", "Trigger", "Session")

tukey_subject = tukey_posthoc(df, "Vividness", "Subject")
tukey_trigger = tukey_posthoc(df, "Vividness", "Trigger")
tukey_session = tukey_posthoc(df, "Vividness", "Session")

print("One-way ANOVA - Subject")
print(anova_subject)
print("\nOne-way ANOVA - Trigger")
print(anova_trigger)
print("\nOne-way ANOVA - Session")
print(anova_session)

print("\nTwo-way ANOVA - Subject & Trigger")
print(anova_subject_trigger)
print("\nTwo-way ANOVA - Subject & Session")
print(anova_subject_session)
print("\nTwo-way ANOVA - Trigger & Session")
print(anova_trigger_session)

print("\nTukey Post-hoc - Subject")
print(tukey_subject)
print("\nTukey Post-hoc - Trigger")
print(tukey_trigger)
print("\nTukey Post-hoc - Session")
print(tukey_session)


/opt/anaconda3/envs/BCI/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 25, but rank is 19
  warnings.warn('covariance of constraints does not have full '
/opt/anaconda3/envs/BCI/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 25, but rank is 19
  warnings.warn('covariance of constraints does not have full '
/opt/anaconda3/envs/BCI/lib/python3.10/site-packages/scipy/integrate/_quadpack_py.py:1272: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


One-way ANOVA - Subject
                 sum_sq      df           F  PR(>F)
C(Subject)  4466.972339    25.0  239.537925     0.0
Residual    6567.927219  8805.0         NaN     NaN

One-way ANOVA - Trigger
                  sum_sq      df         F        PR(>F)
C(Trigger)    130.125213    26.0  4.040652  2.198366e-11
Residual    10904.774345  8804.0       NaN           NaN

One-way ANOVA - Session
                  sum_sq      df          F    PR(>F)
C(Session)     18.230483     1.0  14.610309  0.000133
Residual    11016.669075  8829.0        NaN       NaN

Two-way ANOVA - Subject & Trigger
                            sum_sq      df           F         PR(>F)
C(Subject)             4466.859009    25.0  288.842613   0.000000e+00
C(Trigger)              130.011883    26.0    8.083673   2.796620e-30
C(Subject):C(Trigger)  1409.419503   650.0    3.505306  4.909203e-150
Residual               5028.495833  8129.0         NaN            NaN

Two-way ANOVA - Subject & Session
                 

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

df = pd.read_csv("Trigger_Vividness_Data.csv")

def three_way_anova(data, dv, iv1, iv2, iv3):
    model = smf.ols(f"{dv} ~ C({iv1}) * C({iv2}) * C({iv3})", data=data).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    return anova_table

anova_results = three_way_anova(df, "Vividness", "Subject", "Trigger", "Session")

print("\nThree-way ANOVA - Subject, Trigger, and Session")
print(anova_results)
